In [3]:
import websocket, json, pprint, talib, numpy as np
import config
from binance.client import Client
from binance.enums import *
import math

In [4]:
SOCKET = "wss://stream.binance.com:9443/ws/xrpusdt@kline_1m"

In [13]:
RSI_Period = 9
TRADE_SYMBOL = 'XRPUSDT'
BASE ='USDT'
ALT='XRP'

In [6]:
closes = []
in_position = True

In [7]:
client = Client(config.API_KEY, config.API_SECRET)

In [111]:
def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET,price='0.0001',timeInForce=TIME_IN_FORCE_GTC):
    try:
        print("sending order")
        if order_type == ORDER_TYPE_MARKET:
            order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        elif order_type == ORDER_TYPE_LIMIT:
            order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity,price=price,timeInForce=timeInForce)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

In [8]:
def on_open(ws):
    print('opened connection')

In [9]:
def on_close(ws):
    print('closed connection')

In [17]:
def on_error(ws, error):
    print(error)

In [70]:
def on_message(ws, message):
    global closes, in_position
    
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)
    
    candle = json_message['k']
    
    is_candle_closed = candle['x']
    close = candle['c']
    
    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        #print(closes)
        print("The length of closes is {}".format(len(closes)))
        print("The value of SMA_HIGH is {}".format(SMA_HIGH))
        
        if len(closes) > SMA_HIGH:
            print("Step 1")
            np_closes = numpy.array(closes)
            RSI=talib.(np_closes,timeperiod=SMA_HIGH)
            SMA_L=talib.SMA(np_closes,timeperiod=SMA_LOW)
            print("The value of SMA_H is {}".format(SMA_H))
            print("The value of SMA_L is {}".format(SMA_L))
            
            last_sma_h = SMA_H[-1]
            last_sma_l = SMA_L[-1]
            
            last_sma_l_2 = SMA_L[-2]
            last_sma_l_3 = SMA_L[-3]
            
            last_sma_h_2 = SMA_H[-2]
            last_sma_h_3 = SMA_H[-3]
            
            print(last_sma_h)
            print(last_sma_l)
            
            if last_sma_l < last_sma_h and last_sma_l_2 < last_sma_h_2 and last_sma_l_3 < last_sma_h_3:
                if in_position:
                    print("Nothing to Do, Already Bought")
                else:
                    if last_sma_l > last_sma_l_2 and last_sma_l_2 < last_sma_l_3:
                        TRADE_QUANTITY=float(client.get_asset_balance(asset=BASE)['free'])//np_closes[-1]
                        order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = True
                            print("BUY Order Placed and succeded")
                        else:
                            print("BUY Order placed but failed")
                        
                        
            if last_sma_l > last_sma_h and last_sma_l_2 > last_sma_h_2 and last_sma_l_3 > last_sma_h_3:
                if in_position:
                    if last_sma_l < last_sma_l_2 and last_sma_l_2 > last_sma_l_3:
                        TRADE_QUANTITY=math.floor(float(client.get_asset_balance(asset=ALT)['free'])*10)/10
                        order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                        if order_succeeded:
                            in_position = False
                            print("SELL ORDER Placed and succeeded")
                        else:
                            print("SELL Order placed but failed")
                else:
                    print('Do Nothing','Already Sold')

In [19]:
def on_message(ws, message):
    global closes, in_position
    json_message = json.loads(message)
    candle = json_message['k']
    #pprint.pprint(json_message)
    is_candle_closed = candle['x']
    close = candle['c']
    
    if is_candle_closed:
        global RSI_COUNTER=0
        RSI_HIST = []
        print("candle closed at {}".format(close))
        closes.append(float(close))
        np_closes = np.array(closes)
        if len(closes) > RSI_Period:
            print(np_closes)
            #print(np_closes[-4:])
            RSI=talib.RSI(np_closes,timeperiod=RSI_Period)
            print("priniting RSI[-1]")
            print(RSI[-1])
    
    elif not is_candle_closed and RSI[-1] >65:
        diff_l=[]
        print("closes_sell")
        closes_sell = closes[-10:]
        print(closes_sell)
        closes_sell.append(float(close))
        np_closes_sell =np.array(closes_sell)
        print('Printing closes sell')
        print(np_closes_sell)
        RSI_SELL = talib.RSI(np_closes_sell,timeperiod=RSI_Period)
        RSI_HIST.append(RSI_SELL[-1])
        RSI_COUNTER += 1
        print("printing RSI_HIST")
        print(RSI_HIST)
        print("printing RSI_SELL[-1]")
        print(RSI_SELL[-1])
        for i in range(len(RSI_HIST-10),len(RSI_HIST)):
            prev = RSI_HIST[i-1]
            curr = RSI_HIST[i]
            diff_l.append(prev-curr)
            diff =sum(diff_l)
        print("printing diff")
        print(diff)
        if RSI_SELL[-1] > RSI[-1]:
            if RSI[-1]>70:
                if RSI_COUNTER > 10 and diff <0:
                    print("Time to SELL Since previous RSI is {} and current RSI is {}".format(RSI[-1],RSI_SELL[-1]))
            elif RSI_SELL[-1] > 70 and RSI[-1] <70:
                if RSI_COUNTER > 10 and diff < 0:
                    print("Time to SELL Since previous RSI is {} and current RSI is {}".format(RSI[-1],RSI_SELL[-1]))
                       
    elif not is_candle_closed and RSI[-1] < 35:
        diff_l=[]
        print("closes_buy")
        closes_buy = closes[-10:]
        print(closes_buy)
        closes_buy.append(float(close))
        np_closes_buy = np.array(closes_buy)
        print('Printing closes buy')
        print(np_closes_buy)
        RSI_BUY = talib.RSI(np_closes_buy,timeperiod=RSI_PERIOD)
        RSI_HIST.append(RSI_BUY[-1])
        RSI_COUNTER += 1
        print("printing RSI_HIST")
        print(RSI_HIST)
        print("printing RSI_BUY[-1]")
        print(RSI_BUY[-1])
        for i in range(len(RSI_HIST-10),len(RSI_HIST)):
            prev = RSI_HIST[i-1]
            curr = RSI_HIST[i]
            diff_l.append(prev-curr)
            diff =sum(diff_l)
        print("printing diff")
        print(diff)
        if RSI_BUY[-1] < RSI[-1]:
            if RSI[-1]<30:
                if RSI_COUNTER > 10 and diff >0:
                    print("Time to BUY since previous RSI is {} and current RSI is {}".format(RSI[-1],RSI_BUY[-1]))
            elif RSI_BUY[-1] < 30 and RSI[-1]>30:
                if RSI_COUNTER > 10 and diff >0:
                    print("Time to BUY since previous RSI is {} and current RSI is {}".format(RSI[-1],RSI_BUY[-1]))
        

In [23]:
def on_message(ws, message):
    global closes, in_position,RSI_COUNTER, last_RSI, RSI_HIST,ALT_BALANCE,BASE_BALANCE,buy_counter,sell_counter,diff,last_sell_RSI
    json_message = json.loads(message)
    candle = json_message['k']
    #pprint.pprint(json_message)
    is_candle_closed = candle['x']
    close = candle['c']
    
    if is_candle_closed:                         
        RSI_COUNTER=0
        RSI_HIST = []
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)
        ALT_BALANCE = float(client.get_asset_balance(asset=ALT)['free'])
        BASE_BALANCE = float(client.get_asset_balance(asset=BASE)['free'])
        print(ALT_BALANCE ,BASE_BALANCE)
        sell_counter = math.floor((ALT_BALANCE*float(close)/12))
        buy_counter = math.floor((BASE_BALANCE)/12)
        print("sell counter is {}".format(str(sell_counter)))
        print("buy counter is {}".format(str(buy_counter)))
        np_closes = np.array(closes)
        if len(closes) > RSI_Period:
            print(np_closes)
            #print(np_closes[-4:])
            RSI=talib.RSI(np_closes,timeperiod=RSI_Period)
        print("RSI Counter is {}".format(str(RSI_COUNTER)))
        print("priniting last_RSI")
        last_RSI=RSI[-1]
        print(last_RSI)
        
        
    if not is_candle_closed:
        RSI_COUNTER += 1
#         print("RSI Counter is {}".format(str(RSI_COUNTER)))
#         print("Candle is not  closed at {}".format(close))
#         print("priniting RSI[-1]")
#         print(last_RSI)
        diff_l=[]
        closes_cur = closes[-50:]
        closes_cur.append(float(close))
        np_closes_cur =np.array(closes_cur)
#         print('Printing closes cur')
#         print(np_closes_cur)
        
        if last_RSI >65:
            RSI_SELL = talib.RSI(np_closes_cur,timeperiod=RSI_Period)
            RSI_HIST.append(RSI_SELL[-1])
            print("printing RSI_HIST")
            print(RSI_HIST[-10:])
            print("printing RSI_SELL[-1]")
            last_sell_RSI = RSI_SELL[-1]
            print("last_RSI is {}".format(str(last_RSI)))
            print("last_sell_RSI is {}".format(str(last_sell_RSI)))
            if RSI_COUNTER > 10:
                for i in range(len(RSI_HIST)-10,len(RSI_HIST)):
                    prev = RSI_HIST[i-1]
                    curr = RSI_HIST[i]
                    diff_l.append(curr-prev)
                    diff =sum(diff_l)
                print("printing diff")
                print(diff)
                if last_sell_RSI > last_RSI:
                    print("last_RSI is {} and RSI_COUNTER is {} and diff is {} and sell_counter is {} ".format(str(last_RSI),str(RSI_COUNTER),str(diff),str(sell_counter)))
                    if last_RSI > 70:
                        if diff < 0 and sell_counter > 0:
                            print("Time to SELL Since previous RSI is {} and current RSI is {} and diff is {}".format(last_RSI,last_sell_RSI,diff))
                            TRADE_SELL_QUANTITY = math.floor(((ALT_BALANCE)/(((ALT_BALANCE)*float(close))/12))*10)/10
                            print("TRADE_SELL_QUANTITY is {}".format(str(TRADE_SELL_QUANTITY)))
                            order_succeeded = order(SIDE_SELL, TRADE_SELL_QUANTITY, TRADE_SYMBOL)
                            if order_succeeded:
                                in_position = False
                                sell_counter -= 1
                                print("SELL ORDER Placed and succeeded")
                            else:
                                print("SELL Order placed but failed")
                    elif last_sell_RSI > 70 and last_RSI < 70:
                        if diff < 0 and sell_counter > 0:
                            print("Time to SELL Since previous RSI is {} and current RSI is {} and diff is {}".format(last_RSI,last_sell_RSI,diff))
                            TRADE_SELL_QUANTITY = math.floor(((ALT_BALANCE)/(((ALT_BALANCE)*float(close))/12))*10)/10
                            print("TRADE_SELL_QUANTITY is {}".format(str(TRADE_SELL_QUANTITY)))
                            order_succeeded = order(SIDE_SELL, TRADE_SELL_QUANTITY, TRADE_SYMBOL)
                            if order_succeeded:
                                in_position = False
                                sell_counter -= 1
                                print("SELL ORDER Placed and succeeded")
                            else:
                                print("SELL Order placed but failed")
        elif last_RSI < 35:
            RSI_BUY = talib.RSI(np_closes_cur,timeperiod=RSI_Period)
            RSI_HIST.append(RSI_BUY[-1])
            #RSI_COUNTER += 1
#             print("printing RSI_HIST")
#             print(RSI_HIST)
#             print("printing RSI_BUY[-1]")
#             print(RSI_BUY[-1])
            if RSI_COUNTER > 10:
                for i in range(len(RSI_HIST)-10,len(RSI_HIST)):
                    prev = RSI_HIST[i-1]
                    curr = RSI_HIST[i]
                    diff_l.append(curr-prev)
                    diff =sum(diff_l)
#                 print("printing diff")
#                 print(diff)
                if RSI_BUY[-1] < last_RSI:
                    print("last_RSI is {} and RSI_COUNTER is {} and diff is {} and buy_counter is {} ".format(str(last_RSI),str(RSI_COUNTER),str(diff),str(buy_counter)))
                    if last_RSI < 30:
                        if diff >0 and buy_counter > 0:
                            print("Time to BUY since previous RSI is {} and current RSI is {} and diff is {}".format(last_RSI,RSI_BUY[-1],diff))
                            TRADE_BUY_QUANTITY=float(12)//float(close)
                            print("TRADE_BUY_QUANITY is {}".format(str(TRADE_BUY_QUANTITY)))
                            order_succeeded = order(SIDE_BUY, TRADE_BUY_QUANTITY, TRADE_SYMBOL)
                            if order_succeeded:
                                in_position = True
                                print("BUY Order Placed and succeded")
                                buy_counter -=1
                            else:
                                print("BUY Order placed but failed")
                            
                    elif RSI_BUY[-1] < 30 and last_RSI>30:
                        if diff >0 and buy_counter > 0:
                            print("Time to BUY since previous RSI is {} and current RSI is {} and diff is {}".format(last_RSI,RSI_BUY[-1],diff))
                            TRADE_BUY_QUANTITY=float(12)//float(close)
                            print("TRADE_BUY_QUANITY is {}".format(str(TRADE_BUY_QUANTITY)))
                            order_succeeded = order(SIDE_BUY, TRADE_BUY_QUANTITY, TRADE_SYMBOL)
                            if order_succeeded:
                                in_position = True
                                print("BUY Order Placed and succeded")
                                buy_counter -=1
                            else:
                                print("BUY Order placed but failed")

In [24]:
closes = []
in_position = True
RSI_COUNTER=0
last_RSI =0
RSI_HIST=[]
sell_counter =0
buy_counter= 0
last_sell_RSI = 0
diff =0

In [25]:
websocket.enableTrace(True)
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message,on_error = on_error)
ws.run_forever()

--- request header ---
GET /ws/xrpusdt@kline_1m HTTP/1.1
Upgrade: websocket
Host: stream.binance.com:9443
Origin: http://stream.binance.com:9443
Sec-WebSocket-Key: deaVkjozHAh85hv+6CW3Qg==
Sec-WebSocket-Version: 13
Connection: upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Sun, 07 Mar 2021 16:16:10 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: aiD8MuQxChvpxOPqrC6uupqFi5A=
-----------------------


opened connection
candle closed at 0.46508000
closes
[0.46508]


error from callback <function on_message at 0x0000018595AEA700>: local variable 'RSI' referenced before assignment


60.9823 0.12930267
sell counter is 2
buy counter is 0
RSI Counter is 0
priniting last_RSI


  File "C:\Users\Dinesh\anaconda3\lib\site-packages\websocket\_app.py", line 346, in _callback
    callback(self, *args)
  File "<ipython-input-23-90a858ab66e3>", line 30, in on_message
    last_RSI=RSI[-1]


...

send: b'\x8a\x8d\x0e\x9f\xa0\xb3?\xa9\x91\x86?\xab\x94\x86?\xaf\x92\x8b7'


candle closed at 0.46086000
closes
[0.46508, 0.46506, 0.46498, 0.46521, 0.46471, 0.46492, 0.46478, 0.46472, 0.46471, 0.46486, 0.46455, 0.46437, 0.46405, 0.46421, 0.46507, 0.46505, 0.46508, 0.46475, 0.46484, 0.46502, 0.46462, 0.46484, 0.46474, 0.46485, 0.46457, 0.46495, 0.4648, 0.46476, 0.4649, 0.46548, 0.46566, 0.46568, 0.46611, 0.4657, 0.46521, 0.4653, 0.46505, 0.46497, 0.46472, 0.46503, 0.46453, 0.46478, 0.46488, 0.4651, 0.46509, 0.46508, 0.46458, 0.46529, 0.46531, 0.46534, 0.46538, 0.46534, 0.46549, 0.46521, 0.46518, 0.46523, 0.46532, 0.46544, 0.46515, 0.46563, 0.46569, 0.46577, 0.46563, 0.46541, 0.46555, 0.46542, 0.46532, 0.46533, 0.46549, 0.46571, 0.46626, 0.4662, 0.46619, 0.46629, 0.46586, 0.46585, 0.46599, 0.46589, 0.4658, 0.4659, 0.46634, 0.4664, 0.46623, 0.466, 0.46586, 0.46593, 0.46603, 0.46602, 0.46576, 0.46586, 0.46542, 0.46546, 0.46542, 0.46535, 0.46504, 0.46506, 0.46513, 0.46559, 0.46553, 0.4653, 0.46514, 0.46532, 0.46578, 0.46599, 0.46539, 0.46511, 0.46508, 0.46487, 0.46

send: b'\x88\x828\x02i\x86;\xea'



closed connection


False

In [31]:
math.floor(float(client.get_asset_balance(asset=ALT)['free'])*10)/10

58.9

In [32]:
TRADE_QUANTITY=math.floor(float(client.get_asset_balance(asset=ALT)['free'])*10)/10

In [42]:
order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL,ORDER_TYPE_LIMIT,price='0.57')

sending order
{'symbol': 'XRPUSDT', 'orderId': 1735352129, 'orderListId': -1, 'clientOrderId': 'kMRqt0kg8QmTXOdGMBC5qU', 'transactTime': 1613814820156, 'price': '0.57000000', 'origQty': '58.90000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'SELL', 'fills': []}


In [4]:
candlesticks = client.get_historical_klines("ZILUSDT",Client.KLINE_INTERVAL_15MINUTE,"1 day ago UTC")

In [47]:
closes=[]
is_candle_closed = False
close = "0.45636"

In [43]:
RSI_COUNTER

4

In [48]:
if not is_candle_closed and RSI_COUNTER < 5:
    print("Candle is not  closed at {}".format(close))
    diff_l=[]
    closes_sell = closes[-10:]
    closes_sell.append(float(close))
    np_closes_sell =np.array(closes_sell)
    print('Printing closes sell')
    print(np_closes_sell)
    RSI_COUNTER += 1
    print("RSI Counter is {}".format(str(RSI_COUNTER)))

Candle is not  closed at 0.45636
Printing closes sell
[0.45636]
RSI Counter is 5


In [7]:
for candle in candlesticks:
    closes.append(float(candle[4]))
print(closes)

[0.13977, 0.13929, 0.1391, 0.139, 0.13951, 0.13942, 0.13917, 0.13942, 0.1395, 0.13969, 0.13948, 0.13948, 0.13946, 0.13925, 0.13967, 0.13967, 0.13981, 0.13968, 0.13952, 0.13959, 0.14001, 0.13994, 0.13993, 0.13974, 0.14007, 0.14003, 0.14002, 0.14009, 0.13996, 0.13967, 0.13977, 0.13948, 0.13955, 0.13828, 0.13832, 0.13812, 0.13777, 0.13748, 0.13775, 0.13803, 0.13833, 0.13798, 0.13764, 0.13724, 0.13721, 0.13779, 0.13782, 0.13751, 0.1373, 0.13761, 0.1379, 0.13784, 0.13799, 0.13794, 0.13862, 0.1383, 0.13846, 0.13823, 0.13833, 0.13872, 0.13858, 0.13866, 0.13838, 0.13838, 0.13898, 0.13908, 0.13903, 0.13886, 0.13905, 0.13919, 0.13958, 0.13955, 0.13961, 0.13948, 0.13955, 0.13953, 0.13934, 0.13931, 0.13941, 0.13939, 0.13959, 0.1396, 0.13983, 0.13935, 0.13974, 0.13967, 0.13973, 0.14, 0.13961, 0.13959, 0.1398, 0.13985, 0.13967, 0.13962, 0.13928, 0.13947, 0.1394, 0.13951, 0.1395, 0.13978, 0.1399, 0.13963, 0.13965, 0.1399, 0.13986, 0.13926, 0.13921, 0.13911, 0.13932, 0.13955, 0.13938, 0.13911, 0.13912

In [8]:
del closes[-1]

In [9]:
print(closes)

[0.13977, 0.13929, 0.1391, 0.139, 0.13951, 0.13942, 0.13917, 0.13942, 0.1395, 0.13969, 0.13948, 0.13948, 0.13946, 0.13925, 0.13967, 0.13967, 0.13981, 0.13968, 0.13952, 0.13959, 0.14001, 0.13994, 0.13993, 0.13974, 0.14007, 0.14003, 0.14002, 0.14009, 0.13996, 0.13967, 0.13977, 0.13948, 0.13955, 0.13828, 0.13832, 0.13812, 0.13777, 0.13748, 0.13775, 0.13803, 0.13833, 0.13798, 0.13764, 0.13724, 0.13721, 0.13779, 0.13782, 0.13751, 0.1373, 0.13761, 0.1379, 0.13784, 0.13799, 0.13794, 0.13862, 0.1383, 0.13846, 0.13823, 0.13833, 0.13872, 0.13858, 0.13866, 0.13838, 0.13838, 0.13898, 0.13908, 0.13903, 0.13886, 0.13905, 0.13919, 0.13958, 0.13955, 0.13961, 0.13948, 0.13955, 0.13953, 0.13934, 0.13931, 0.13941, 0.13939, 0.13959, 0.1396, 0.13983, 0.13935, 0.13974, 0.13967, 0.13973, 0.14, 0.13961, 0.13959, 0.1398, 0.13985, 0.13967, 0.13962, 0.13928, 0.13947, 0.1394, 0.13951, 0.1395, 0.13978, 0.1399, 0.13963, 0.13965, 0.1399, 0.13986, 0.13926, 0.13921, 0.13911, 0.13932, 0.13955, 0.13938, 0.13911, 0.13912